In [4]:
import speech_recognition as sr
from pydub import AudioSegment

/Users/rohankalluraya/miniconda3/envs/defects_env/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
!pip install pydub
!pip install openai
!pip install pydub
!pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 1.1 MB/s eta 0:00:0000:0100:01


In [10]:
!pip install -q -U google-genai

In [5]:
# Function to convert MP3 to WAV
def convert_mp3_to_wav(mp3_file, wav_file):
    audio = AudioSegment.from_mp3(mp3_file)
    audio.export(wav_file, format="wav")

In [30]:
import json

In [31]:
# Function to transcribe audio and extract the medical information

from google import genai

def extract_medical_info(audio_file):
  # Convert MP3 to WAV (if necessary)
    if audio_file.endswith(".mp3"):
        wav_file = "converted_audio.wav"
        convert_mp3_to_wav(audio_file, wav_file)
        audio_file = wav_file

    # Initialize recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    # Recognize speech using Google Web Speech API
    try:
        text = recognizer.recognize_google(audio_data)
        print("Transcription: ", text)
    
        client = genai.Client(api_key="YOUR API KEY")
        response = client.models.generate_content(
            model="gemini-2.0-flash", contents="Analyze the following text, which is a conversation between a patient and a doctor and return it itself in JSON format with the keys 'Patient Name', 'age', 'Symptoms', and 'Doctors Suggestion'. Make sure to strictly use only the text and not add additional information " + text
            )
        # print(response.text)
        lines = response.text.splitlines()
        json_lines = [line for line in lines if not line.strip().startswith("```")]
        clean_json_str = "\n".join(json_lines)

        # Convert to a Python dictionary to verify it's valid JSON
        data = json.loads(clean_json_str)

        # Save to a file
        with open('patient_data.json', 'w') as file:
            json.dump(data, file, indent=4)


    
    except sr.UnknownValueError:
        print("Could not understand the audio")
        return {}
    except sr.RequestError:
        print("Request failed, check your network connection")
        return {}

    # Define keywords for different symptoms
    # symptoms = {
    #     "Pain Descriptions": ["burning", "stabbing", "pressure"],
    #     "Respiratory Issues": ["shortness of breath", "coughing"],
    #     "Neurological Issues": ["dizziness", "numbness", "headache"],
    #     "Infection Signs": ["fever", "chills", "swelling"]
    # }

    # # Create a dictionary to store matched symptoms
    # medical_info = {key: [] for key in symptoms}

    # # Check the transcribed text for each symptom keyword
    # for category, keywords in symptoms.items():
    #     for keyword in keywords:
    #         if keyword in text.lower():
    #             medical_info[category].append(keyword)

    # # Filter out categories with no matches
    # medical_info = {category: values for category, values in medical_info.items() if values}

    return (response.text)


In [32]:
# Usage example
audio_file = "dizz.wav"  # Replace with your audio file path
medical_info = extract_medical_info(audio_file)
print("Extracted Information:", medical_info)

Transcription:  I have dizziness
Extracted Information: ```json
{
  "Patient Name": null,
  "age": null,
  "Symptoms": "I have dizziness",
  "Doctors Suggestion": null
}
```
